In [1]:
import json

# 数据整理

In [2]:
import pandas as pd

In [28]:
train = pd.read_csv('C:/Users/yuki/Desktop/大创/数据/LCQMC/LCQMC/LCQMC_train.csv')
test = pd.read_csv('C:/Users/yuki/Desktop/大创/数据/LCQMC/LCQMC/LCQMC_test.csv')
dev = pd.read_csv('C:/Users/yuki/Desktop/大创/数据/LCQMC/LCQMC/LCQMC_dev.csv')

In [29]:
data=list(train['sentence1'])+list(test['sentence1'])+list(dev['sentence1'])

In [30]:
data=list(set(data))

In [31]:
f=open('C:/Users/yuki/Desktop/大创/数据/原句子.txt','w',encoding='utf-8')
for line in data:
    f.write(line+'\n')

In [32]:
f=open('C:/Users/yuki/Desktop/大创/数据/原句子.txt','r',encoding='utf-8')
data=[]
lines = f.readlines()      #读取全部内容 ，并以列表方式返回
for line in lines:
    line=line.replace('\n','')
    line=line.replace('\t','')
    data.append(line)

In [33]:
len(data)

159523

In [34]:
data[:10]

['男肚子疼,胃也疼还拉肚子是怎么回事',
 '最近昆明天气怎么样？',
 '武媚娘在入寺怀的是谁的孩子',
 '最快的速度(成语)',
 '完成下列光路图、',
 '八年级寒假作业答案全部',
 '这个酒多少钱一瓶？',
 '南方和北方哪个好',
 '百度浏览器怎么样？',
 '请问大侠门,做一个刚才销售人员必备的东西是什么']

### 是否需要去除非法字符？

# 抽取

In [15]:
import thulac

In [16]:
thu1 = thulac.thulac() #默认模式

Model loaded succeed


# 句子模板抽取-词性抽取

n/名词 np/人名 ns/地名 ni/机构名 nz/其它专名

m/数词 q/量词 mq/数量词 t/时间词 f/方位词 s/处所词

v/动词 a/形容词 d/副词 h/前接成分 k/后接成分 i/习语 

j/简称 r/代词 c/连词 p/介词 u/助词 y/语气助词

e/叹词 o/拟声词 g/语素 w/标点 x/其它

## 抽取 名词+习语+形容词

书名号里的不能提取

In [17]:
#第一种词性标注
data_extra=[]
for i in data[:50000]:
    text=thu1.cut(i,text=True) #进行一句话分词
    data_extra.append(text)

In [18]:
f=open('C:/Users/yuki/Desktop/大创/数据/data_extra.txt','w',encoding='utf-8')
for line in data_extra:
    f.write(line+'\n')

In [19]:
data_extra=[]
for line in open('C:/Users/yuki/Desktop/大创/数据/data_extra.txt',encoding='UTF-8'):
    line=line.rstrip().split('\t')
    line=line[0].strip().split()
    data_extra.append(line)

In [20]:
#按照词性标注1挖句子模板
data_extra_dic=[]
for item in data_extra:
    temp={}
    lst=[]
    extra=[]
    for i in item:
        if ('_n' == i[-2:]) or ('_i' == i[-2:]) or ('_a' == i[-2:]):
            lst.append('moudle')
            extra.append(i.split('_'))
        else:
            lst.append(i.split('_')[0])
    temp['原句']=item
    temp['抽取的句子']=lst
    temp['抽取的词']=extra
    data_extra_dic.append(temp)

In [140]:
#第二种词性标注
data_extra2=[]
for i in data[:50000]:
    text=HanLP.segment(i)
    test=text.toArray()
    temp=[]
    for i in test:
        temp.append(i.toString())
    data_extra2.append(temp)

In [145]:
f=open('C:/Users/yuki/Desktop/大创/数据/data_extra2.txt','w',encoding='utf-8')
for line in data_extra2:
    for i in line:
        f.write(i+' ')
    f.write('\n')

In [21]:
data_extra2=[]
for line in open('C:/Users/yuki/Desktop/大创/数据/data_extra2.txt',encoding='UTF-8'):
    line=line.rstrip().split('\t')
    line=line[0].strip().split()
    data_extra2.append(line)

In [22]:
#按照词性标注2挖句子模板
data_extra_dic2=[]
for item in data_extra2:
    temp={}
    lst=[]
    extra=[]
    for i in item:
        if ('/n' == i[-2:]) or ('/i' == i[-2:]) or ('/a' == i[-2:]) or ('/vn' == i[-3:]):
            lst.append('moudle')
            extra.append(i.split('/'))
        else:
            lst.append(i.split('/')[0])
    temp['原句']=item
    temp['抽取的句子']=lst
    temp['抽取的词']=extra
    data_extra_dic2.append(temp)

In [23]:
filename='C:/Users/yuki/Desktop/大创/数据/data_extra_dic.json'
with open(filename,'w',encoding='utf-8') as file_obj:
    json.dump(data_extra_dic,file_obj,ensure_ascii=False,indent = 4)

In [24]:
filename='C:/Users/yuki/Desktop/大创/数据/data_extra_dic2.json'
with open(filename,'w',encoding='utf-8') as file_obj:
    json.dump(data_extra_dic2,file_obj,ensure_ascii=False,indent = 4)

# 句子模板抽取-句法依存关系抽取

## 抽取 动宾关系的宾语+并列关系中的名词、形容词、成语+定中关系的名词、形容词、成语

#### 使用粗粒度的标签作为词性标签

1. ID：代币计数器，每个新句子从1开始。
2. FORM：单词形式或标点符号 
3. LEMMA：词形的词组或词干
4. CPOSTAG：粗粒度的语料部分标签 
5. POSTAG：细粒度的言语部分标签 
6. FEATS：无序的句法和/或形态学特征集合，用竖条"|"分隔，如果没有，则用下划线"_" 
7. HEAD：当前标记的头部ID 
8. DEPREL：与HEAD的依存关系
--
1. ID: token counter, starting at 1 for each new sentence 
2. FORM: word form or punctuation symbol 
3. LEMMA: lemma or stem of word form
4. CPOSTAG: coarse-grained part-of-speech tag 
5. POSTAG: fine-grained part-of-speech tag 
6. FEATS: unordered set of syntactic and/or morphological features, separated by a vertical bar “|”, or an underscore “_” if not available 
7. HEAD: head ID of the current token 
8. DEPREL: dependency relation to the HEAD

词性参考：http://www.hankcs.com/nlp/part-of-speech-tagging.html#h2-8

动宾关系中 尾部的词
并列关系中 n a i
定中关系中 n a i

In [68]:
from pyhanlp import *

In [17]:
#句法分析
data_para=[]
for i in data[:50000]:
    text=HanLP.parseDependency(i)
    data_para.append(text)

In [28]:
#整理
data_para_dic=[]
for item in data_para:
    temp={}
    temp['首']=[]
    temp['关系']=[]
    temp['尾']=[]
    temp['切分']=[]
    for word in item.iterator():
        temp['首'].append([word.ID,word.CPOSTAG])
        temp['关系'].append(word.DEPREL)
        temp['尾'].append([word.HEAD.ID,word.HEAD.CPOSTAG])
        temp['切分'].append(word.LEMMA)
        
    data_para_dic.append(temp)

In [32]:
filename='C:/Users/yuki/Desktop/大创/数据/data_para_dic.json'
with open(filename,'w',encoding='utf-8') as file_obj:
    json.dump(data_para_dic,file_obj,ensure_ascii=False,indent = 4)

In [25]:
with open('C:/Users/yuki/Desktop/大创/数据/data_para_dic.json',encoding="utf-8") as file_obj:
    data_para_dic=json.load(file_obj)

In [26]:
#按照句法分析挖句子模板
for d in range(0,len(data_para_dic)):
    dic=data_para_dic[d]
    rela=dic['关系']
    head=dic['首']
    tail=dic['尾']
    seg=dic['切分']
    extra=[]
    #动宾关系的宾语
    for num in range(0,len(rela)):
        if rela[num]=='动宾关系':
            h=head[num]
            t=tail[num]
            if h[0]>t[0]:
                extra.append(h)
            elif h[0]<t[0]:
                extra.append(t)
        if rela[num]=='并列关系':
            h=head[num]
            t=tail[num]
            if h[1]=='n' or h[1]=='a' or h[1]=='i' or h[1]=='vn':
                extra.append(h)
            if t[1]=='n' or t[1]=='a' or t[1]=='i' or t[1]=='vn':
                extra.append(t)
        if rela[num]=='定中关系':
            h=head[num]
            t=tail[num]
            if h[1]=='n' or h[1]=='a' or h[1]=='i' or h[1]=='vn':
                extra.append(h)
            if t[1]=='n' or t[1]=='a' or t[1]=='i' or t[1]=='vn':
                extra.append(t)
                
    data_para_dic[d]['extra']=extra

In [36]:
#按照句法分析挖句子模板
data_para_extra_dic=[]
for d in range(0,len(data_para_dic)):
    dic=data_para_dic[d]
    temp={}
    temp['原句子']=dic['切分']
    temp['原句子词性']=dic['首']
    lst=dic['切分'][:]
    word=[]
    count=[]
    for i in dic['extra']:
        if i[0] in count:
            pass
        else:
            word.append([lst[i[0]-1],i[1]])
            lst[i[0]-1]='moudle'
            count.append(i[0])
        
    temp['抽取的句子']=lst
    temp['抽取的词']=word
    data_para_extra_dic.append(temp)

In [37]:
filename='C:/Users/yuki/Desktop/大创/数据/data_para_extra_dic111.json'
with open(filename,'w',encoding='utf-8') as file_obj:
    json.dump(data_para_extra_dic,file_obj,ensure_ascii=False,indent = 4)

In [12]:
with open('C:/Users/yuki/Desktop/大创/数据/data_para_extra_dic.json',encoding="utf-8") as file_obj:
    data_para_extra_dic=json.load(file_obj)

In [38]:
with open('C:/Users/yuki/Desktop/大创/数据/data_extra_dic2.json',encoding="utf-8") as file_obj:
    test=json.load(file_obj)

In [39]:
len(test)

49890

In [40]:
with open('C:/Users/yuki/Desktop/大创/数据/data_para_extra_dic111.json',encoding="utf-8") as file_obj:
    test11=json.load(file_obj)

In [41]:
len(test11)

50000